In [ ]:
# default_exp pivoting

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# pivoting

Um was gehts
- Wie sollen die Daten aufbereitet werden, so dass wir möglichst gut ergänzen können?
- möghlichst wenig daten?
- möglichst alle notwendigen Informationen
- möglichst ideal gruppiert

Idee:
mehrere pivotierte Tabellen (immer pro Statement typ)
- 1. mit Werten
- 2. mit Statement
- 3. mit Zeile im Statement

Alle Sätze separat ablegen.. 
vorerst alle Attribute mitnehmen, die existieren

Statement types

- IS: IncomeStatement
- CF: CashFlow
- BS: BalanceSheet
- CI: Comprehensive Income
- EQ: Equity
- CP: CoverPage
- UN: Unclassifiable Statement

Erwartete  Grössen:
-   IS   4559669
-   CF   4435322
-   BS   4821120
-   CI    850503
-   EQ    899115
-   CP     13408
-   UN     71711

## Basic Settings

In [ ]:
# imports
from bfh_mt_hs2020_sec_data.core import get_spark_session # initialze spark
from pathlib import Path
from typing import List, Tuple, Union, Set
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col

import pandas as pd

import shutil          # provides high level file operations
import time            # used to measure execution time
import os
import sys

In [ ]:
all_filtered_folder        = "D:/data/parq_filtered"
all_pivot_selected_folder  = "D:/data/parq_pivot_select"
all_pivoted_folder         = "D:/data/parq_pivot_split"

In [ ]:
# init Spark
spark = get_spark_session() # Session anlegen
spark # display the moste important information of the session

## 01_Load Data

**only execute if necessary**

In [ ]:
df_all = spark.read.parquet(all_filtered_folder).cache()

In [ ]:
# Expected Amount of Data
# Entries:  15_650_848
# duration:  66 sec

# load all data into memory
start = time.time()
print("Entries: ", "{:_}".format(df_all.count())) # loading all dataset into memory
duration = time.time() - start
print("duration: ", duration)

Entries:  15_650_848
duration:  96.77243518829346


In [ ]:
p_df_prim_fin_rep_count = df_all.cube('stmt').count().toPandas()
print(p_df_prim_fin_rep_count)

   stmt     count
0    IS   4559669
1  None  15650848
2    UN     71711
3    CF   4435322
4    CP     13408
5    BS   4821120
6    CI    850503
7    EQ    899115


## 02_Select

only keep relevant columns

**only execute if necessary**

In [ ]:
df_all_selected = df_all.select(["stmt","cik","ticker", "adsh","period","form","tag","value","report", "line", "fp", "uom"])

In [ ]:
shutil.rmtree(all_pivot_selected_folder,  ignore_errors=True)
df_all_selected.write.parquet(all_pivot_selected_folder)

In [ ]:
# produziert 1.5 GB CSV Datei unkomprimiert
# df_all_selected.toPandas().to_csv("./data/pivot_select.csv",index=False,header=True)

## 03_Pivoting

In [ ]:
df_all_selected = spark.read.parquet(all_pivot_selected_folder).cache()

In [ ]:
# Expected Amount of Data
# Entries:  15_650_848
# duration:  23 sec

# load all data into memory
start = time.time()
print("Entries: ", "{:_}".format(df_all_selected.count())) # loading all dataset into memory
duration = time.time() - start
print("duration: ", duration)

Entries:  15_650_848
duration:  16.718958616256714


In [ ]:
pivot_attrs = ['value', 'report', 'line']

def pivot_statement(all_data_df, statement:str):
    all_stmt_data = all_data_df.where("stmt == '" + statement + "'").cache()
    
    shutil.rmtree(all_pivoted_folder + "/" + statement,  ignore_errors=True)
    
    grouped_df = all_stmt_data.groupby(["cik","ticker","adsh","form","period","fp"])
    
    for attr in pivot_attrs:
        pivoted_df = grouped_df.pivot("tag").max(attr)
        pivoted_df.repartition(1).write.parquet(all_pivoted_folder + "/" + statement + "/" + attr)
        
    all_stmt_data.unpersist()

In [ ]:
def pivot_statements(all_data_df, statements):
    for stmt in statements:
        print (stmt)
        pivot_statement(all_data_df, statement=stmt)

## 99_Execution

In [ ]:
#statements = ['IS','CF','CP','BS','CI','EQ','UN']
statements = ['UN']
pivot_statements(df_all_selected, statements)

UN


AttributeError: 'GroupedData' object has no attribute 'unpersist'

In [ ]:
spark.stop()

In [ ]:
df_all_selected.where("stmt = 'UN'").select("cik","ticker","adsh","form","period","fp").show()